<a href="https://colab.research.google.com/github/ManuelResendiz/LasActividadesVanAqui/blob/main/pia_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificador de reseñas
Inteligencia artificila que detecta comentarios negativos y positivos de una base de datos de reseñas de peliculas. A continuacion se vere el procedimiento. Se basa en el Análisis de los sentimientos paea dectarar si son parrafos positivo o negativos , los va indicar con 0 y 1(negativo y positivos).

Primero se instala las librerias correspondientes

In [ ]:
!pip install tensorflow-hub
!pip install tensorflow-datasets

In [ ]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.8.0
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


## Descagar los datos a usar en el" the IMDB dataset"



In [ ]:

train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)
# Divida el conjunto de entrenamiento en 60% y 40% para terminar con 15,000 ejemplos
# para entrenamiento, 10,000 ejemplos para validación y 25,000 ejemplos para prueba.

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete3LFAG8/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete3LFAG8/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


#prueba de datos
La etiqueta es un valor entero de 0 o 1, donde 0 es una reseña negativa y 1 es una reseña positiva.

Imprimamos los primeros 10 ejemplos.

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

Aqui vemos que las primeras etiquetas ya nos arrojo si son positivas (1) y negativas (0) que vendrian siendo las reseñas de prueba que en este caso seleccionamos 10.

In [ ]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

# Construccion del modelo

Primero, creemos una capa de Keras que use un modelo de TensorFlow Hub para incrustar las oraciones y pruébalo en un par de ejemplos de entrada. Tenga en cuenta que, independientemente de la longitud del texto de entrada, la forma de salida de las incorporaciones es: (num_examples, embedding_dimension) .



In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

# **`Modelo `**

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


Un modelo necesita una función de pérdida y un optimizador para el entrenamiento. Como se trata de un problema de clasificación binaria y el modelo genera logits (una capa de una sola unidad con una activación lineal), utilizará la función de pérdida binary_crossentropy .

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Entrenar el modelo

Entrene el modelo durante 10 épocas en mini lotes de 512 muestras. Son 10 iteraciones sobre todas las muestras en los tensores x_train y y_train . Mientras entrena, controle la pérdida y la precisión del modelo en las 10 000 muestras del conjunto de validación:

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 21s 647ms/step - loss: 0.6384 - accuracy: 0.5703 - val_loss: 0.5781 - val_accuracy: 0.6729
Epoch 2/10
30/30 [==============================] - 20s 636ms/step - loss: 0.5048 - accuracy: 0.7436 - val_loss: 0.4694 - val_accuracy: 0.7695
Epoch 3/10
30/30 [==============================] - 20s 638ms/step - loss: 0.3748 - accuracy: 0.8469 - val_loss: 0.3858 - val_accuracy: 0.8357
Epoch 4/10
30/30 [==============================] - 20s 646ms/step - loss: 0.2728 - accuracy: 0.8981 - val_loss: 0.3371 - val_accuracy: 0.8543
Epoch 5/10
30/30 [==============================] - 20s 652ms/step - loss: 0.2009 - accuracy: 0.9311 - val_loss: 0.3202 - val_accuracy: 0.8512
Epoch 6/10
30/30 [==============================] - 20s 642ms/step - loss: 0.1492 - accuracy: 0.9526 - val_loss: 0.3111 - val_accuracy: 0.8735
Epoch 7/10
30/30 [==============================] - 19s 634ms/step - loss: 0.1089 - accuracy: 0.9701 - val_loss: 0.3071 - val_accuracy: 0.8724

## Evualar el modelo

Veamos cómo se comporta el modelo. Se devolverán dos valores. Pérdida (un número que representa nuestro error, los valores más bajos son mejores) y precisión.

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 5s - loss: 0.3626 - accuracy: 0.8517 - 5s/epoch - 101ms/step
loss: 0.363
accuracy: 0.852


Este Modelo tiene una presicion del 85% bastante aceptable, este portejate puede aumentar con otro modelo mas comlejo 
